In [ ]:
!pip install torch
!pip install transformers

In [ ]:
!pip install diffgram
!pip install llama_index
!pip install matplotlib
!pip install tabulate

In [4]:
import json
from diffgram import Project
from llama_index.core import SimpleDirectoryReader, StorageContext
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib as mpl
from tabulate import tabulate
import math

/tmp/ipykernel_8527/2932230743.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [5]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connect to diffgram project

In [6]:
project_local = Project(host = "https://7ea5-2604-3d08-4f7f-e8c0-64d7-690b-93d-9368.ngrok-free.app",
        project_string_id = "valiantbiter",
        client_id = "LIVE__ywmyayoir4c6zx1kz5j4",
        client_secret = "wwwh2eiwc7f4q35qywxgghuyfuh2oemwshec0tiu4gctapsxqe3orcyfe5u3")

## Utility functions to create/check a directory or file in diffgram with dataset_name

In [7]:
def check_if_directory_exist(dir_name):
    project = project_local.directory.get_directory_list(limit=50000)
    for project_dir in project:
        if (project_dir.__dict__['nickname'] == dir_name):
            return project_dir
    return None
    #if (project_dir.__dict__['nickname'])

In [8]:
## You may need to run this twice to see if the directory is created
def create_diffgram_directory(dataset_name):
    #directory = project_local.directory.get(name = dataset_name)
    directory = check_if_directory_exist(dataset_name)
    if (directory is None):
        project_local.directory.new(name=dataset_name)
        directory = check_if_directory_exist(dataset_name)
        print(directory.__dict__)    
    return directory

In [9]:
## check if file exist in the dir
def check_if_file_exist_in_dir(filename):
    file = project_local.file.file_list_exists(filename)

In [10]:
def create_dataset_job(data_suffix, job_suffix, index, member_list_ids):
    dataset_batch_name = data_suffix + str(index)
    directory = create_diffgram_directory(dataset_batch_name)

    if directory is None:
        print(f"{dataset_batch_name} Directory does not exist")
        return

    job_name = job_suffix + str(index)

    job = project_local.job.new(
        name = job_name,
        instance_type = "box",
        share = "Project",
        sync_directories = [directory],
        label_schema_id = schema_id,
        tag_list = ["Laws", "Acts", "Regulations"],
        members_list_ids = member_list_ids,
        auto_launch = True
    )
    print(f"The {job_name} task is created")
    return directory

In [11]:
def upload_files_to_dataset(index,batch_size, offset, directory):
    for document in range((index+offset) * batch_size, ((index + offset) * batch_size) + batch_size):
        filename = diffgram_documents[document].metadata['filename']
        # check if the file exist in the diffgram directory
        try:
            file = project_local.file.from_local(filename,directory_id=directory.__dict__['id'])
        except:
            print(f"File with {filename} exist in this directory. Continuing ....")
            continue;

## Import all the files 
### make sure you have the diffgram_processing_v2 folder which has all the data arranged for NER task

In [12]:
from llama_index.core import SimpleDirectoryReader, StorageContext

In [13]:
file_metadata = lambda x: {"filename": x}
diffgram_documents = SimpleDirectoryReader("diffgram_processing_v2",file_metadata=file_metadata).load_data()

In [14]:
print(len(diffgram_documents))

81612


In [15]:
print(diffgram_documents[0].get_text())

Chunk ID: Criminal Records Review Act-chunk-Reconsideration-0003
Act ID: Criminal Records Review Act
Regulation ID: None
Section Name: Reconsideration
Section ID: 5
Sequence ID: 3
Text:
7 the registrar must promptly provide notification of a decision under subsection ( 6 ) ( a ) or ( b ) to the individual who is the subject of the decision and to the persons or entities that were provided with a notification under section 4 ( 4. 1 ) or ( 4. 2 ) or 4. 1 ( 4 ), as applicable.



## Fetch and add the schema

In [18]:
f = open('legal_ner_schema.json')
NER_schema = json.load(f)
print(NER_schema)

{'schema': [{'name': 'O', 'description': 'Outside of named entity (default)', 'example': 'The'}, {'name': 'B-INTERNAL_REF_SECTION', 'description': 'Beginning of internal section reference', 'example': "Section in 'Section 101'"}, {'name': 'I-INTERNAL_REF_SECTION', 'description': 'Inside of internal section reference', 'example': "101 in 'Section 101'"}, {'name': 'B-EXTERNAL_REF_ACT_SECTION', 'description': 'Beginning of external act section reference', 'example': "Section in 'Section 5 of the Clean Air Act'"}, {'name': 'I-EXTERNAL_REF_ACT_SECTION', 'description': 'Inside of external act section reference', 'example': "5 in 'Section 5 of the Clean Air Act'"}, {'name': 'B-EXTERNAL_REF_REGULATION_SECTION', 'description': 'Beginning of external regulation section reference', 'example': "Section in 'Section 3.2 of the Federal Acquisition Regulation'"}, {'name': 'I-EXTERNAL_REF_REGULATION_SECTION', 'description': 'Inside of external regulation section reference', 'example': "3.2 in 'Section 

In [19]:
## Id the Labels
# Create label to ID mapping
label2id = []
id2label = []
for id, label in enumerate(NER_schema['schema']):
    label2id.append(id)
    id2label.append(NER_schema['schema'][id]['name'])
print(label2id)
print(id2label)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
['O', 'B-INTERNAL_REF_SECTION', 'I-INTERNAL_REF_SECTION', 'B-EXTERNAL_REF_ACT_SECTION', 'I-EXTERNAL_REF_ACT_SECTION', 'B-EXTERNAL_REF_REGULATION_SECTION', 'I-EXTERNAL_REF_REGULATION_SECTION', 'B-REF_TYPE', 'I-REF_TYPE', 'B-EXCLUDED_SECTION', 'I-EXCLUDED_SECTION', 'B-INCLUDED_SECTION', 'I-INCLUDED_SECTION', 'B-DECISION', 'I-DECISION', 'B-ACT_NAME', 'I-ACT_NAME', 'B-REGULATION_NAME', 'I-REGULATION_NAME', 'B-SUBSECTION', 'I-SUBSECTION', 'B-PARAGRAPH', 'I-PARAGRAPH']


In [ ]:
NER_schema_name = 'NER_TRAINING_SCHEMA'
schema_id = None
schemas = project_local.schema.list()
print(json.dumps(schemas, indent=2))
for schema in schemas:
    if schema['name'] == NER_schema_name:
        schema_id = schema['id']
if schema_id == None:
    json_response = project_local.new_schema(name=NER_schema_name)
    schema_id = json_response["id"]
    for NER in NER_schema['schema']:
        print(NER['name'])
        project_local.label_new(NER, schema_id=schema_id)

## Fetch the files and add it to diffgram
if you want to insert one file at a time
    file = project_local.file.from_local("diffgram_processing_v2/800.txt")

### Separating the data into train and test
1. Batch each train or test data to a size of 64 or any multiple of 2^ (easier for memory alignment/ faster processing)
2. To make the netowkr good at NER the more data it is trained on the better. IF we want to train it with 5000 examples, we need to create 5000/64 batches of test data
3. We can then have 5% * 5000 as the test data
4. we can name the diffgram dataset as NER_train_batch_x where x is the batch number
5. similar we can name the diffgram dataset for test as NER_test_batch_x 
6. After creating the dataset we create the task for both test and train e.g. NER_train_JOB_. 
7. The task links the dataset and the schema.
8. After which we add files to the dataset based on the batch size
9. Repeat steps 4 - 8 till all the data is populated and arranged for annotation

In [151]:
def create_datasets(diffgram_documents, num_training_data, batch_size, train_suffix, test_suffix, job_train_suffix, job_test_suffix):
    #check if the lenght of all the data is equal to or more than num_training_data
    if (len(diffgram_documents) < num_training_data):
        print(f"Not sufficient data for training {len(diffgram_documents)}")
        return
    
    train_batch_size = math.floor(num_training_data/batch_size)
    test_batch_size = math.floor((num_training_data * (5 /100))/ batch_size)
    #train_dataset_name = "NER_train_batch_"
    
    print(f"The batch size of the training data is : {train_batch_size}")
    print(f"The batch size of the test data is: {test_batch_size}")
    
    member_list = project_local.get_member_list()
    member_list_ids = [x['member_id'] for x in member_list]
    
    #schemas = project_local.schema.list()
    
    for index in range(0, train_batch_size):
        directory = create_dataset_job(train_suffix, job_train_suffix, index, member_list_ids)
        
        print(f"Creating / Uploading data to directory {directory.__dict__['nickname']}")
        upload_files_to_dataset(index,batch_size, 0, directory)
        
    for index in range(0, test_batch_size):
        directory = create_dataset_job(test_suffix, job_test_suffix, index, member_list_ids)
        
        print(f"Creating / Uploading data to directory {directory.__dict__['nickname']}")
        upload_files_to_dataset(index,batch_size, train_batch_size+1, directory)

In [20]:
NUM_TRAINING_DATA = 5000
BATCH_SIZE = 64
train_dataset_suffix = "NER_train_batch_"
test_dataset_suffix = "NER_test_batch_"
JOB_NAME = "Law_NER_task1"
JOB_TRAIN_SUFFIX = "NER_train_JOB_"
JOB_TEST_SUFFIX = "NER_test_JOB_"

In [ ]:
create_datasets(diffgram_documents, NUM_TRAINING_DATA, BATCH_SIZE, train_dataset_suffix, test_dataset_suffix,  JOB_TRAIN_SUFFIX, JOB_TEST_SUFFIX)

## Export annotated data

In [21]:
def get_job_names(diffgram_documents, num_training_data, batch_size, data_suffix):
    if (len(diffgram_documents) < num_training_data):
        print(f"Not sufficient data for training {len(diffgram_documents)}")
        return
    
    num_batches = math.floor(num_training_data/batch_size)
    jobs = []
    for index in range(0, num_batches):
        jobs_name = JOB_TRAIN_SUFFIX + str(index)
        jobs.append(jobs_name)
    return jobs

In [22]:
jobs = get_job_names(diffgram_documents, NUM_TRAINING_DATA, BATCH_SIZE, train_dataset_suffix)

In [23]:
get_job = project_local.job.list(limit=10000, page_number=1)

In [24]:
print(len(get_job))

60


In [25]:
## Get the job based on the id and name
def get_completed_job(jobs):
    get_job = project_local.job.list()
    #print(f"Number of jobs {len(get_job)}")
    completed_jobs = []
    for jobs_completed in get_job:
        #print(f"{jobs_completed['status']} {jobs_completed['name']}")
        for job in jobs:
            #print(f"Job name is {job}")
            if (jobs_completed['status'] == 'complete') and (jobs_completed['name'] == job):
                print(f"{jobs_completed['name']} is completed")   
                completed_jobs.append(jobs_completed)
                break
            else:
                #print(f"{job} is not annotated completely. Please comeplete annotation to train the A.I") 
                continue
    return completed_jobs            
        #print(f"{jobs_completed['name']} is not completed")    

In [26]:
completed_jobs = get_completed_job(jobs)
#print(completed_jobs)
if completed_jobs is not None:
    for completed_job in completed_jobs:
        print(f"The status of the job {completed_job['name']} (id = {completed_job['id']})  is {completed_job['status']}.")
else: 
    print("The annotation task is not completed. Complete all annotation before proceeding.")

NER_train_JOB_0 is completed
The status of the job NER_train_JOB_0 (id = 23)  is complete.


In [27]:
results = project_local.job

In [28]:
#def download_annotated_data(completed_jobs):
#    for completed_job in completed_jobs:
#        results.refresh_from_dict(completed_job)

In [29]:
results.refresh_from_dict(completed_jobs[0])

In [30]:
completed_annotations = results.generate_export()

In [31]:
DIFFGRAM_EXPORT_ADDITIONAL_PARAMETERS = 5

In [32]:
print(f" Number of annotated data {len(completed_annotations) - DIFFGRAM_EXPORT_ADDITIONAL_PARAMETERS}")

 Number of annotated data 64


## Post Processing
### processing the annotated data to train the A.I

In [33]:
data = []
sentences = []
labels = []
data_index = 0
for completed_annotation in completed_annotations:
    #print(f"{completed_annotation} ----")
    if (completed_annotation != 'attribute_groups_reference')  \
        and (completed_annotation != 'export_info') \
        and (completed_annotation != 'label_map') \
        and (completed_annotation != 'readme') \
        and (completed_annotation != 'label_colour_map'):
        sentence_local = []
        labels_local = []
        # First get the point where the annotation is started
        for start in completed_annotations[completed_annotation]['instance_list']:
            if 'start_token' in start:
                start_token =  start['start_token']
                break
            
        #start_token = completed_annotations[completed_annotation]['instance_list'][0]['start_token']
        for annotated_index in range(start_token, len(completed_annotations[completed_annotation]['text']['tokens']['words'])):
            # check if this text is annotated
            for data in completed_annotations[completed_annotation]['instance_list']:
                if 'start_token' in data:
                    if annotated_index == data['start_token']:
                        sentence_local.append(completed_annotations[completed_annotation]['text']['tokens']['words'][annotated_index]['value'])
                        labels_local.append(completed_annotations['label_map'][str(data['label_file_id'])])
                        #print(f"{completed_annotations[completed_annotation]['text']['tokens']['words'][annotated_index]['value']} - {completed_annotations['label_map'][str(data['label_file_id'])]}")
                        break;
        sentences.append(sentence_local)       
        labels.append(labels_local)
        data_index+=1

In [34]:
len(sentences)

64

## View of the annotated data before feeding into the neural network for training

In [42]:
# Create DataFrame
# Creating a pandas DataFrame
for iter in range (0, 1):
    pd.set_option('display.max_colwidth', None)
    ner_df = pd.DataFrame({
        "Token": sentences[iter],
        "Label": labels[iter]
    })  
    # Display the DataFrame with some custom styles
    ner_df.style.set_properties(**{'background-color': 'lightyellow', 
                               'color': 'black',
                               'border-color': 'black'})
    # Use tabulate to display a nice table in the terminal
    print(tabulate(ner_df, headers='keys', tablefmt='fancy_grid'))

╒════╤══════════════╤════════════════════════╕
│    │ Token        │ Label                  │
╞════╪══════════════╪════════════════════════╡
│  0 │ 7            │ O                      │
├────┼──────────────┼────────────────────────┤
│  1 │ the          │ O                      │
├────┼──────────────┼────────────────────────┤
│  2 │ registrar    │ O                      │
├────┼──────────────┼────────────────────────┤
│  3 │ must         │ O                      │
├────┼──────────────┼────────────────────────┤
│  4 │ promptly     │ O                      │
├────┼──────────────┼────────────────────────┤
│  5 │ provide      │ O                      │
├────┼──────────────┼────────────────────────┤
│  6 │ notification │ O                      │
├────┼──────────────┼────────────────────────┤
│  7 │ of           │ O                      │
├────┼──────────────┼────────────────────────┤
│  8 │ a            │ O                      │
├────┼──────────────┼────────────────────────┤
│  9 │ subsec

## Preparing the data for input to pytorch
### We need to make all the data the same length by padding if they are not 255 tokens in length

In [47]:
# Load pre-trained tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

/opt/app-root/lib64/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [44]:
MAX_LENGTH = 255
PAD_TOKEN = "[PAD]"
PAD_LABEL = "O"

In [45]:
def pad_sequence(sequence, max_length, pad_value):
    return sequence + [pad_value] * (max_length - len(sequence))

In [48]:
# Create DataFrame
data = []
NER_label = []
for sent_id, (sentence, sent_labels) in enumerate(zip(sentences, labels)):
    tmp_data = []
    tokens = sentence  # Simple tokenization
    #print(tokens)
    sentence_string = ""
    for string_value in tokens:
        sentence_string += string_value + ' '
    #print(sentence_string)
    padded_tokens = pad_sequence(tokens, MAX_LENGTH, PAD_TOKEN)
    padded_labels = pad_sequence(sent_labels, MAX_LENGTH, PAD_LABEL)
    encoded = tokenizer(sentence_string,
                        padding='max_length',
                        truncation=True,
                        max_length=MAX_LENGTH,  # Adjust based on your needs
                        return_tensors='pt')
    #print(encoded['input_ids'][0][0])
    data.append(encoded)
    NER_label.append(padded_labels)
    for token_id in range(MAX_LENGTH):
        tmp_data.append({
            'Sentence_ID': sent_id,
            'Token_ID': token_id,
            'Token': padded_tokens[token_id],
            'NER_Label': padded_labels[token_id],
            'Is_Pad': padded_tokens[token_id] == PAD_TOKEN
        })
    #data.append(tmp_data)

In [49]:
def convert_label_to_id(ner_label,id2label):
    label_id = []
    for label in ner_label:
        label_id.append(id2label.index(label))
    return label_id

## Training the model

In [50]:
# Load pre-trained model
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)

# Training loop (simplified)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
device = "cpu"

In [ ]:
for epoch in range(5):  # Number of epochs
    for i, batch in enumerate(data):
        labelled_id = convert_label_to_id(NER_label[i], id2label)
        #print(labelled_id)
        input_ids = torch.tensor(batch['input_ids']).to(device)
        attention_mask = torch.tensor(batch['attention_mask']).to(device)
        labels = torch.tensor(labelled_id, dtype=torch.long).to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

## Evaluating the model

In [72]:
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
test_data = data
with torch.no_grad():
    for i, batch in enumerate(test_data):
        # Convert inputs to tensors
        input_ids = torch.tensor(batch['input_ids']).to(device)
        attention_mask = torch.tensor(batch['attention_mask']).to(device)
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        print(predictions)
        
        # Compare predictions with true labels if available
        #true_labels = batch['labels']  # Assuming test data has true labels
        # Calculate accuracy or other metrics

## Lets compare the accuracy with the trained dataset. See if it is overfitting or underfitting

In [76]:
def compare_output():
    print("Comparing the models predictions vs the labelled predictions")

In [77]:
compare_output()

Comparing the models predictions vs the labelled predictions


## Post-Processing NER Output

### Steps

1. **Find `B-REF-TYPE`**
   - Look for the token labeled as `B-REF_TYPE`, which indicates the beginning of a reference type like "subsection" or "paragraph."

2. **Check if it is Followed by `B-INTERNAL_REF_SECTION` or `B-EXTERNAL_REF_ACT_SECTION/REGULATION_SECTION`**
   - After finding `B-REF_TYPE`, check the subsequent tokens to see if they are labeled as `B-INTERNAL_REF_SECTION`, `B-EXTERNAL_REF_ACT_SECTION`, or `B-EXTERNAL_REF_REGULATION_SECTION`.
   - This step ensures that the reference type is correctly associated with either an internal or external reference.

3. **Handle Internal References**
   - If `B-INTERNAL_REF_SECTION` is found, capture all tokens labeled with `I-INTERNAL_REF_SECTION` until the first `O` (Outside of named entity) is encountered.
   - Convert the captured tokens into a section number or identifier.
   - Create a Cypher query to establish a link to the internal reference in your graph database.

4. **Handle External References**
   - If `B-EXTERNAL_REF_ACT_SECTION` or `B-EXTERNAL_REF_REGULATION_SECTION` is found, capture all tokens labeled with `I-EXTERNAL_REF_ACT_SECTION` or `I-EXTERNAL_REF_REGULATION_SECTION` until the first `O` is encountered.
   - Continue to search for tokens labeled `B-ACT_NAME` or `B-REGULATION_NAME`, which indicate the act or regulation name.
   - Capture these tokens and combine them with the section number to form a full reference.
   - Create a Cypher query to establish a link to the external reference, specifying the act or regulation name.


In [72]:
# Create DataFrame
# Creating a pandas DataFrame
references = []
for iter in range (0, 1):
    length_of_sentence = len(sentences[iter])
    for index, sentence in enumerate(sentences[iter]):
        #print(f"{sentence} {labels[iter][index]}")
        
        #Step 1: Check B-REF-TYPE
        if (labels[iter][index] == 'B-REF_TYPE'):
            ref_type = labels[iter][index]
            ref_content = ""
            ref_target = ""
            
            #Step 2: Check for the next label
            if ((index+1) > (length_of_sentence - 1)):
                continue
            j = index +1    
            next_label = labels[iter][j]
            
            # Step 3: Handle Internal References
            if next_label.startswith('B-INTERNAL_REF_SECTION'):
                ref_content += sentences[iter][j] + " "
                j+=1
                while (j < length_of_sentence) and labels[iter][j].startswith('I-INTERNAL_REF_SECTION'):
                    print(sentences[iter][j])
                    ref_content += sentences[iter][j] + " "
                    j += 1
                ref_content = ref_content.strip()
                references.append({"type": "internal", "ref_type": ref_type, "content": ref_content})
                print(references)
                # Generate Cypher query
                cypher_query = f"CREATE (n:InternalRef {{type: '{ref_type}', section: '{ref_content}'}})"
                print(cypher_query)
                
            # Step 4: Handle External References
            elif next_label.startswith('B-EXTERNAL_REF'):
                ref_content += sentences[iter][j] + " "
                j+=1
                while (j < length_of_sentence) and labels[iter][j].startswith('I-INTERNAL_REF_SECTION'):
                    print(sentences[iter][j])
                    ref_content += sentences[iter][j] + " "
                    j += 1
                #ref_content = ref_content.strip()
                references.append({"type": "internal", "ref_type": ref_type, "content": ref_content})
                
                # Search for ACT_NAME or REGULATION_NAME
                while j < len(ner_output):
                    if ner_output[j][1] == 'B-ACT_NAME' or ner_output[j][1] == 'B-REGULATION_NAME':
                        k = j
                        while k < len(ner_output) and ner_output[k][1].startswith(('I-ACT_NAME', 'I-REGULATION_NAME')):
                            ref_target += ner_output[k][0] + " "
                            k += 1
                        ref_target = ref_target.strip()
                        break
                    j += 1
                references.append({"type": "external", "ref_type": ref_type, "content": ref_content, "target": ref_target})
                print(references)

                # Generate Cypher query
                cypher_query = f"CREATE (n:ExternalRef {{type: '{ref_type}', section: '{ref_content}', target: '{ref_target}'}})"
                print(cypher_query)

    # Print the data in a pretty table
    pd.set_option('display.max_colwidth', None)
    ner_df = pd.DataFrame({
        "Token": sentences[iter],
        "Label": labels[iter]
    })  
    # Display the DataFrame with some custom styles
    ner_df.style.set_properties(**{'background-color': 'lightyellow', 
                               'color': 'black',
                               'border-color': 'black'})
    # Use tabulate to display a nice table in the terminal
    print(tabulate(ner_df, headers='keys', tablefmt='fancy_grid'))

[{'type': 'internal', 'ref_type': 'B-REF_TYPE', 'content': '4'}]
CREATE (n:InternalRef {type: 'B-REF_TYPE', section: '4'})
╒════╤══════════════╤════════════════════════╕
│    │ Token        │ Label                  │
╞════╪══════════════╪════════════════════════╡
│  0 │ 7            │ O                      │
├────┼──────────────┼────────────────────────┤
│  1 │ the          │ O                      │
├────┼──────────────┼────────────────────────┤
│  2 │ registrar    │ O                      │
├────┼──────────────┼────────────────────────┤
│  3 │ must         │ O                      │
├────┼──────────────┼────────────────────────┤
│  4 │ promptly     │ O                      │
├────┼──────────────┼────────────────────────┤
│  5 │ provide      │ O                      │
├────┼──────────────┼────────────────────────┤
│  6 │ notification │ O                      │
├────┼──────────────┼────────────────────────┤
│  7 │ of           │ O                      │
├────┼──────────────┼──────────

# ------------------------------------------------------------------------------------------------
# **************************************  End of Script  *****************************************
# ------------------------------------------------------------------------------------------------